<a href="https://colab.research.google.com/github/jao208/side_file_random_generator/blob/main/Randomisation_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Please execute the first cell below by clicking the play button**
You will be asked to provide API token for the study as well as a sample URL to a randomisation instrument for any subjects.

Once you have obtained these, click "Submit" and execute the second cell.

In [1]:
import requests
import pandas as pd
import numpy as np
import json
from collections import defaultdict
import panel as pn
pn.extension()
import ipywidgets as widgets
from IPython.display import clear_output
from datetime import datetime
import time
import os

####----#### Specify Parameters ####----####
token_widget = widgets.Text( value='', placeholder='', description='API Token:',disabled=False)
url_widget = widgets.Text( value='', placeholder='', description='Sample URL:', disabled=False )
submit_button = widgets.Button(description="Submit")
display(token_widget,url_widget)

def on_button_clicked(b):
    clear_output(wait=False)
    print("---> Now execute the code below <---")

submit_button.on_click(on_button_clicked)
widgets.VBox([submit_button])

---> Now execute the code below <---


In [2]:
# API Code
# https://euuat.redcapcloud.com/#cid=nph2008&act=view&studyId=981&id=35550&expandedSEDIds=4287_1

rcc_link = url_widget.value      
my_token = token_widget.value 
rcc_link = rcc_link.strip()
locator_env = ["eu","redcapcloud.com"]
locator_stid = ["studyId","&"]
locator_event = ["expandedSEDIds","_"]
rcc_envr = rcc_link[rcc_link.find(locator_env[0]):rcc_link.find(locator_env[1])-1]
fromID = rcc_link[rcc_link.find(locator_stid[0])+len(locator_stid[0])+1:]
fromEvent = rcc_link[rcc_link.find(locator_event[0])+len(locator_event[0])+1:]
studyId = int(fromID[0:fromID.find(locator_stid[1])])
eventDefinitionId = int(fromEvent[0:fromEvent.find(locator_event[1])])

####----#### Get subject IDs and event-crf data ####----####
my_headers = {'token':my_token}
r_sub = requests.get('https://'+rcc_envr+'.redcapcloud.com/rest/v2/subjects', headers=my_headers)
r_evtcrf = requests.get('https://'+rcc_envr+'.redcapcloud.com/rest/v2/event-crfs', headers=my_headers)                    
response_sub_json=r_sub.json()
response_evtcrf_json=r_evtcrf.json()

df_sub = pd.read_json(json.dumps(response_sub_json))[['uniqueIdentifier','id']].rename(columns={'id':"subjectId_sub"}).sort_values(by=['subjectId_sub']).reset_index(drop=True)
df_evtcrf_bool = pd.read_json(json.dumps(response_evtcrf_json))['eventDefinitionId']==eventDefinitionId
df_evtcrf = pd.read_json(json.dumps(response_evtcrf_json))[['id','subjectId','studyEventId']][df_evtcrf_bool].sort_values(by=['subjectId']).reset_index(drop=True)
if(len(df_sub)!=len(df_evtcrf)):
  print("Numbers don't agree")
df = pd.concat([df_sub,df_evtcrf],axis=1).sort_values(by='uniqueIdentifier').reset_index(drop=True)

if not all(df['subjectId_sub'].values==df['subjectId'].values):
  print('sorting didin\'t go right sorting')

####----#### Prepare Side file ####----####
str1 = chr(92)+chr(34)+'https://'+rcc_envr+'.redcapcloud.com/' + '#cid=nph2008&act=editEv&studyId='+str(studyId)+'&id='
str2 = '&sjId='
str3 = '&expandedSEIds=&expandedSEDIds='+str(eventDefinitionId)+"_1"+'&expandedRCGIds=&stEventId='

links=pd.DataFrame(str1+df['id'].astype(str)+str2+df['subjectId'].astype(str)+str3+df['studyEventId'].astype(str)+'\\"',columns=['link'])
link_to_side=" ["+links.T.squeeze().str.cat(sep=',')+"];"

before_side = '{ \"id\": \"e8250306-0fcc-4976-be68-6147feb19c89\", \"version\": \"2.0\", \"name\": \"Randomisation testing\", \"url\": \"google.com\", \"tests\": [{ \"id\": \"e696fd75-f8b2-4ef6-82bb-bfd797c60649\", \"name\": \"Untitled\", \"commands\": [{ \"id\": \"9daf32d0-bd32-475c-be0e-3ca872c7beae\", \"comment\": \"\", \"command\": \"executeScript\", \"target\": \"return'
after_side = '\", \"targets\": [], \"value\": \"x\" }, { \"id\": \"9dd2ec67-ed6f-4e86-a654-ff12a6fbf709\", \"comment\": \"\", \"command\": \"forEach\", \"target\": \"x\", \"targets\": [], \"value\": \"linkss\" }, { \"id\": \"436b5a2d-c128-450e-8630-12438b946e2c\", \"comment\": \"\", \"command\": \"open\", \"target\": \"${linkss}\", \"targets\": [], \"value\": \"\" }, { \"id\": \"09d1e817-c450-421a-928d-799651795f66\", \"comment\": \"\", \"command\": \"pause\", \"target\": \"1500\", \"targets\": [], \"value\": \"\" }, { \"id\": \"c3020aa9-956c-4a51-acc7-8ca35f461074\", \"comment\": \"\", \"command\": \"click\", \"target\": \"css=#dataCollectionWidget_buttonRandomize > span\", \"targets\": [ [\"css=#dataCollectionWidget_buttonRandomize > span\", \"css:finder\"], [\"xpath=//button[@id=\'dataCollectionWidget_buttonRandomize\']/span\", \"xpath:idRelative\"], [\"xpath=//div[2]/div/table/tbody/tr/td/table/tbody/tr/td[2]/button/span\", \"xpath:position\"], [\"xpath=//span[contains(.,\'Randomize\')]\", \"xpath:innerText\"] ], \"value\": \"\" }, { \"id\": \"f42fe1f6-a61a-477b-90ef-dc36fe8ab741\", \"comment\": \"\", \"command\": \"pause\", \"target\": \"1000\", \"targets\": [], \"value\": \"\" }, { \"id\": \"58fc3c6c-d1fc-4eb5-9bef-bb3e521ceea9\", \"comment\": \"\", \"command\": \"end\", \"target\": \"\", \"targets\": [], \"value\": \"\" }] }, { \"id\": \"02d5e52a-a6f3-45e5-9b26-b7ed0290ceaa\", \"name\": \"tests\", \"commands\": [] }], \"suites\": [{ \"id\": \"aa942b78-3cb8-488f-bd2d-c68e083ac215\", \"name\": \"Default Suite\", \"persistSession\": false, \"parallel\": false, \"timeout\": 300, \"tests\": [\"e696fd75-f8b2-4ef6-82bb-bfd797c60649\"] }], \"urls\": [], \"plugins\": [] }'
side_file = before_side+link_to_side+after_side
side_file_format=json.loads(side_file)

####----#### Save Side file ####----####
randomisation_script_file_format = open("randomisation_script_format.side","w")
randomisation_script_file_format.write(json.dumps(side_file_format, indent=2,sort_keys=False))
randomisation_script_file_format.close()

# randomisation_script_file = open("randomisation_script.side","w")
# randomisation_script_file.write(side_file)
# randomisation_script_file.close()
pn.extension()
file_download = pn.widgets.FileDownload(file='randomisation_script_format.side', filename='randomisation_script_format_'+time.strftime("%Y_%m_%d_%H_%M_%S")+'.side',width=400)
print('Side file for Selenium IDE is ready to download. Click on the button below to download (make sure your browser allows downloads from this website):')
file_download


Side file for Selenium IDE is ready to download. Click on the button below to download (make sure your browser allows downloads from this website):


FileDownload(file='randomisation_script_form..., filename='randomisation_script_form..., label='Download randomisation_sc..., width=400)